First, load the data,in this notebook we are using the smokoy mountain data as the example. And will use basic AE to get the transformation of the data and do prediction using the same models.

!!! problem, the result is not the same when rerun it (even the number of latent space is not the same)
#The final result ended with a n=50 latent space, and the performance is better then all features

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

2025-01-21 11:35:39.412166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 11:35:39.414721: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 11:35:39.418629: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 11:35:39.428819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737477339.447058 3145304 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737477339.45

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import svm
import pickle
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  0


2025-01-21 11:35:42.052533: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### 1. Load the data and data preprocess
After reading the data, convert the abundance matrix into relative abundance matrix; for the response variables, since we have multiple variables, organize them into an array with each variable as a column.

In [4]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [5]:
data = FS.relative_abundance(data)

### 2.AE

In [6]:
from tensorflow.keras import layers
from tensorflow import keras as ks
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Model

# --------------------------------------------------------------------------------------------------#

# To define your model, use the Keras Model Subclassing API.
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='relu'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded



In [7]:
data_train, data_test, y_train, y_test = RunML_continue.split_and_scale_data(data,burn_label)
print(data_train.shape)
print(data_test.shape)

(41, 1768)
(18, 1768)


In [8]:
data_train.shape[1]

1768

In [9]:
latent_dim = 20
shape = data_train.shape[1:]
print(shape)
autoencoder = Autoencoder(latent_dim, shape)
autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())

(1768,)


In [10]:
autoencoder.fit(data_train, data_train,
                epochs=10,
                shuffle=True,
                validation_data=(data_test, data_test))

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - loss: 0.2002 - val_loss: 1.5739
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1942 - val_loss: 1.5725
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1922 - val_loss: 1.5713
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1916 - val_loss: 1.5702
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.1977 - val_loss: 1.5693
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1989 - val_loss: 1.5686
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1935 - val_loss: 1.5679
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1913 - val_loss: 1.5671
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.1943 - val_loss: 1.5662
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.1935 - val_loss: 1.5652


In [11]:
data_AE = autoencoder.encoder(data).numpy()

In [12]:
iter =30
cls = ["RF","SVM", "CatBoost","NB"]

data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [13]:
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 20)


The  function will print out the accuracy and AUC for each dataset using each classifier, and also will return the y_actual, y_predict, y_predprob for future use.

In [14]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

TBB Warning: The number of workers is currently limited to 0. The request for 47 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



             RF_Accuracy   RF_AUC   SVM_Accuracy  SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.84375      0.845455        
AE            0.678788    0.637616    0.778788    0.78588      0.593939        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.595139     0.509091    0.611111  


In [15]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.677966  0.428571   0.352941    0.809524     0.172957
AE_SVM                0.779661  0.666667   0.470588    0.904762     0.422324
AE_CatBoost           0.593220  0.294118   0.294118    0.714286     0.008403
AE_NB                 0.508475  0.333333   0.705882    0.428571     0.124855


In [16]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.506061    0.628889    0.657576    0.659365      0.521212        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.537778     0.556061    0.641905  


In [17]:
print(metric.metric_sum(dict_cm_un))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.559322  0.500000   0.230769    0.818182     0.060376
AllFeatures_SVM       0.559322  0.500000   0.346154    0.727273     0.079171
AllFeatures_CatBoost  0.525424  0.400000   0.153846    0.818182    -0.037014
AllFeatures_NB        0.610169  0.551724   0.615385    0.606061     0.219913
AE_RF                 0.508475  0.434783   0.384615    0.606061    -0.009491
AE_SVM                0.661017  0.615385   0.615385    0.696970     0.312354
AE_CatBoost           0.525424  0.450000   0.346154    0.666667     0.013446
AE_NB                 0.559322  0.500000   0.692308    0.454545     0.149490


In [18]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.678788    0.660536    0.777273    0.756964      0.628788        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.674464     0.540909    0.568036  


In [19]:
print(metric.metric_sum(dict_cm_dur))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.593220  0.333333   0.090909    0.891892    -0.027517
AllFeatures_SVM       0.508475  0.230769   0.136364    0.729730    -0.156230
AllFeatures_CatBoost  0.542373  0.222222   0.090909    0.810811    -0.132182
AllFeatures_NB        0.508475  0.360000   0.409091    0.567568    -0.022842
AE_RF                 0.677966  0.571429   0.545455    0.756757     0.305226
AE_SVM                0.779661  0.736842   0.636364    0.864865     0.518730
AE_CatBoost           0.627119  0.500000   0.590909    0.648649     0.233334
AE_NB                 0.542373  0.413793   0.545455    0.540541     0.083181


In [20]:
shape = data_train.shape[1:]
print(shape)

(1768,)


### fine tune latent dimension and activation function

In [21]:
param_grid = {
            #'input_dim': [X_train_scaled.shape[1]], 
            'latent_dim': [10, 25, 50, 100],
            'activation': ['relu', 'sigmoid', 'tanh']  
        }


In [22]:
best_score = float('inf')
best_model = None
for activation in param_grid['activation']:
    class Autoencoder(Model):
      def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
          layers.Flatten(),
          layers.Dense(latent_dim, activation=activation),
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(tf.math.reduce_prod(shape).numpy(), activation=activation),
          layers.Reshape(shape)
        ])
    
      def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    for latent_dim in param_grid['latent_dim']:
        autoencoder = Autoencoder(latent_dim, shape)
        autoencoder.compile(optimizer='adam', loss=ks.losses.MeanSquaredError())
        autoencoder.fit(data_train, data_train,
                    epochs=10,
                    shuffle=True,
                    verbose=0,
                    validation_data=(data_test, data_test))
        val_loss = autoencoder.evaluate(data_test, data_test, verbose=0)
        if val_loss < best_score:
            best_score = val_loss
            print(f"activation:{activation},latent_dim:{latent_dim}, val_loss:{val_loss}")
            best_model = autoencoder
        
            

activation:relu,latent_dim:10, val_loss:1.5645126104354858
activation:relu,latent_dim:50, val_loss:1.563591480255127
activation:relu,latent_dim:100, val_loss:1.5625184774398804


In [23]:
data_AE = best_model.encoder(data).numpy()

In [24]:
data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 100)


In [25]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.843750      0.845455        
AE            0.795455    0.860880    0.712121    0.824769      0.728788        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.812037     0.745455    0.676157  


In [26]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.796610  0.727273   0.470588    0.928571     0.464172
AE_SVM                0.711864  0.500000   0.352941    0.857143     0.236376
AE_CatBoost           0.728814  0.533333   0.470588    0.833333     0.316110
AE_NB                 0.745763  0.600000   0.352941    0.904762     0.311079


In [27]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.607576    0.640714    0.624242    0.595238      0.590909        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.585714     0.659091    0.686825  


In [28]:
print(metric.metric_sum(dict_cm_un))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.559322  0.500000   0.230769    0.818182     0.060376
AllFeatures_SVM       0.559322  0.500000   0.346154    0.727273     0.079171
AllFeatures_CatBoost  0.525424  0.400000   0.153846    0.818182    -0.037014
AllFeatures_NB        0.610169  0.551724   0.615385    0.606061     0.219913
AE_RF                 0.610169  0.578947   0.423077    0.757576     0.191947
AE_SVM                0.627119  0.590909   0.500000    0.727273     0.233334
AE_CatBoost           0.593220  0.541667   0.500000    0.666667     0.168443
AE_NB                 0.661017  0.600000   0.692308    0.636364     0.326397


In [29]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.409091    0.329643    0.580303    0.527500      0.442424        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.419821     0.525758    0.443750  


In [30]:
print(metric.metric_sum(dict_cm_dur))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.593220  0.333333   0.090909    0.891892    -0.027517
AllFeatures_SVM       0.508475  0.230769   0.136364    0.729730    -0.156230
AllFeatures_CatBoost  0.542373  0.222222   0.090909    0.810811    -0.132182
AllFeatures_NB        0.508475  0.360000   0.409091    0.567568    -0.022842
AE_RF                 0.406780  0.157895   0.136364    0.567568    -0.306407
AE_SVM                0.576271  0.400000   0.272727    0.756757     0.032744
AE_CatBoost           0.440678  0.238095   0.227273    0.567568    -0.207206
AE_NB                 0.525424  0.400000   0.545455    0.513514     0.057039
